In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
import json
import collections
import copy
import random
import time
import torch
from torch import nn, cuda, optim
from torch.utils.data import DataLoader
device = 'cuda' if cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.cuda.empty_cache()
print(device)

from transformers import (
    BertModel,
    BertForTokenClassification,
    BertTokenizerFast,
    AutoTokenizer,
    AutoModelForTokenClassification,
    get_linear_schedule_with_warmup
)

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    precision_recall_fscore_support,
    classification_report
)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings('ignore')

def generate_random_seed():
    return random.randint(1, 1000)

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
class Config:
    def __init__(self, runs, epochs, batch_size, 
                 num_labels = 3, label_list = ['O', 'B', 'I'], model_checkpoint = "bert-base-uncased", max_len = 128, error_type = 'simple', use_crf = False):
        self.num_labels = num_labels
        self.label_list = label_list
        self.model_checkpoint = model_checkpoint
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.runs = runs
        self.epochs = epochs
        self.batch_size = batch_size
        self.max_len = max_len
        self.error_type = error_type
        self.use_crf = use_crf

In [3]:
""" Tokenize examples in batch
Since the tokenizer may divide each token into two or more subtokens, we must align the new tokens with the original labels.
New subtokens must have the same label than their parent token
Labels may be 0, 1 or 2 for O, B and I labels, respectively, and -100 for complementary tokens, such PAD, SEP, CLS tokens.
Loss functions will ignore labels with value -100, so the loss only considers mistakes at the positions of real input (sub)tokens.
"""
def tokenize_and_align_labels(txts, lbls, config):
    tokenizer, max_len, mapping = config.tokenizer, config.max_len, config.label_list

    tokenized_inputs = tokenizer(txts, is_split_into_words=True,
                                 max_length = max_len, 
                                 padding = 'max_length', 
                                 truncation=True,
                                 return_tensors = 'pt')

    labels = []
    for i, label in enumerate(lbls):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        previous_label = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.            
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(mapping.index(label[word_idx]))
                previous_label = label[word_idx]
            # For the other tokens in a word, we set the label to the current label.
            else:
                new_label = 'O' if previous_label == 'O' else 'I'+previous_label[1:]
                label_ids.append(mapping.index(new_label))
                previous_label = new_label
                
            previous_word_idx = word_idx

        labels.append(label_ids)

    return tokenized_inputs, labels

"""
Return text tokens from the tokenizer given the numeric input ids
"""
def get_tokens_from_ids(input_ids):

    return [tokenizer.convert_ids_to_tokens(tl) for tl in input_ids]

"""
Remove part of predicted sequences corresponding to padding tokens.
"""
def remove_padding_from_predictions(predictions, batch_attention_mask):
    valid_predictions_list = []
    for instance_preds, att_mask in zip(predictions, batch_attention_mask):
        valid = [pred for pred, mask in zip(instance_preds, att_mask) if mask == 1]
        valid_predictions_list.append(valid[1:-1])
        
    return valid_predictions_list

def remove_padding_and_get_tokens(batch_ids, batch_attention_mask):
    valid_ids_list = []
    for instances_ids, att_mask in zip(batch_ids, batch_attention_mask):
        valid = [ids for ids, mask in zip(instances_ids, att_mask) if mask == 1]
        valid_ids_list.append(valid[1:-1])
    
    valid_tokens = get_tokens_from_ids(valid_ids_list)
    return valid_tokens

"""
Maps sequences of integer to sequences of BIO tags
"""
def integer_to_bio(labels, mapping):
    return [[mapping[int(x)] for x in l] for l in labels]

"""
Transforms list of predicted sequences to a flat list of labels.
"""
def flatten_predictions(labels):
    return [j for sub in labels for j in sub]

"""
Generates txt file with tokens, labels and predictions. Estilo FLAiR NLP.
"""
def save_predictions(tokens, labels, predictions, file_path):
    with open(file_path, 'w', encoding = 'utf-8') as nf:

        for tks, lbs, prds in zip(tokens, labels, predictions):
            for tk, lb, pr in zip(tks, lbs, prds):
                nf.write(f"{tk} {lb} {pr}\n")

            nf.write(f"\n") 

"""
loading data
"""
def load_data(df, config):
    
    train_seq_df = df.loc[df['set'] == 'train']
    if 'dev' in df['set'].values:
        val_seq_df = df.loc[df['set'] == 'dev']
    else:
        train_seq_df, val_seq_df = train_test_split(train_seq_df, test_size = 0.1, random_state = 2023)
    test_seq_df = df.loc[df['set'] == 'test']

    train_dataset, val_dataset, test_dataset = SequenceLabelingDataset(train_seq_df, config), \
                                            SequenceLabelingDataset(val_seq_df, config), \
                                            SequenceLabelingDataset(test_seq_df, config)
    
    batch_size = config.batch_size
    train_loader, val_loader, test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True), \
                                            DataLoader(val_dataset, batch_size=batch_size), \
                                            DataLoader(test_dataset, batch_size=batch_size)
        
    return train_loader, val_loader, test_loader            

def clean_batch_elements(batch_input_ids, batch_labels, batch_predictions, tokenizer):
    eval_tokens = []
    eval_labels = []
    eval_predictions = []
    for i in range(len(batch_input_ids)):
        tokens = tokenizer.convert_ids_to_tokens(batch_input_ids[i].tolist())
        labels = batch_labels[i]
        preds = batch_predictions[i]
        filtered_tokens, filtered_labels, filtered_preds = [], [], []
        for tk, lb, pr in zip(tokens, labels, preds):
            if tk not in tokenizer.all_special_tokens:
#                 tk = tk.lstrip('Ġ')
                if tk != '':
                    filtered_tokens.append(tk)
                    filtered_labels.append(lb)
                    filtered_preds.append(pr)
                
        eval_tokens.append(filtered_tokens)
        eval_labels.append(filtered_labels)
        eval_predictions.append(filtered_preds)
        
    return eval_tokens, eval_labels, eval_predictions




"""
Dataset class for sequence labeling
"""
class SequenceLabelingDataset(torch.utils.data.Dataset):
    
    def __init__(self, df, config):
        lb = [x.split() for x in df.labels.values.tolist()]
        txt = [i.split() for i in df.tokens.values.tolist()]
        self.encodings, self.labels = tokenize_and_align_labels(txt, lb, config)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
    
"""
Model for sequence labeling
"""
class SimpleTagger(nn.Module):
    def __init__(self, config):
        super(SimpleTagger, self).__init__()
        self.model_checkpoint = config.model_checkpoint
        self.num_labels = config.num_labels
        self.transf = AutoModelForTokenClassification.from_pretrained(self.model_checkpoint, num_labels = self.num_labels)
    
    def forward(self, input_ids, attention_mask, token_type_ids, labels = None):
        
        if labels is not None: # training
            x = self.transf(input_ids = input_ids, 
                                    token_type_ids = token_type_ids, 
                                    attention_mask = attention_mask, 
                                    labels = labels)
            return x.logits
        else: # inference
            x = self.transf(input_ids = input_ids, 
                                    token_type_ids = token_type_ids, 
                                    attention_mask = attention_mask, 
                                    labels = labels)
            
            return np.argmax(x.logits.detach().cpu().numpy(), axis = 2).tolist()

In [4]:
class CustomLoss():
    def __init__(self, base_loss_fn, init_weights, config):
        super(CustomLoss, self).__init__()
        self.base_loss_fn = base_loss_fn
        self.config = config
        self.initial_error_weights = init_weights # for static weighting
        self.lambdas = init_weights               # for dynamic weighting
        self.current_epoch_errors = np.zeros(5)   # num errors per category in current training epoch
        self.past_epoch_errors = np.zeros(5)      # num errors per category in last training epoch
        self.acumulated_error_epoch = np.zeros(6)
        self.matrix_errors = np.zeros((config.epochs, 5))
        self.type_weighting = 1
        
        
    def get_category_weights(self):
        return self.lambdas
        
    def restart_error_counter(self):
        self.acumulated_error_epoch = np.zeros(6)
        
    def get_instances_from_batch(self, batch_predictions, batch_labels):
        
        batch_predictions = batch_predictions.detach().cpu().numpy()
        batch_labels = batch_labels.detach().cpu().numpy()
        
        instances = []
        for i in range(len(batch_predictions)):
            labels = batch_labels[i]
            preds = batch_predictions[i]
            
            filtered_labels, filtered_preds = [], []
            for l, p in zip(labels, preds):
                if l != -100:
                    filtered_labels.append(l)
                    filtered_preds.append(p)
                    
            instances.append((filtered_labels, filtered_preds))
            
        return instances
    
    def get_arguments_indices(self, sequence):
        indices = []
        start_index = None
        for i, label in enumerate(sequence):
            if label == 1:

                if start_index is None:
                    start_index = i
                else:
                    indices.append((start_index, i))
                    start_index = i

            elif (label == 0) and (start_index is not None):
                indices.append((start_index, i))
                start_index = None

        if start_index is not None:
            indices.append((start_index, len(sequence)))

        return indices
    
    def compute_r_matrix(self, gold_indices, predicted_indices):

        def get_R(gold_argument, predicted_argument):
            (gold_start, gold_end) = gold_argument
            (pred_start, pred_end) = predicted_argument

            intersection_start = max(gold_start, pred_start)
            intersection_end = min(gold_end, pred_end)

            len_intersection_interval = (intersection_end - intersection_start) if intersection_start <= intersection_end else 0
            len_longer_span = max(gold_end - gold_start, pred_end - pred_start)
            if len_longer_span > 0:
                return round((len_intersection_interval / len_longer_span), 3)
            else:
                return 0

        R_matrix = np.zeros((len(gold_indices), len(predicted_indices)), dtype=float)

        for i, gold_argument in enumerate(gold_indices):
            for j, predicted_argument in enumerate(predicted_indices):
                R_matrix[i][j] = get_R(gold_argument, predicted_argument)

        return R_matrix
    
    def categorization_counter(self, matrix, tau, omega):

        def is_split(matrix, column):
            gold_row_index = np.where(column > 0)[0]
            gold_row = matrix[gold_row_index, :]
            positive_values = gold_row[gold_row > 0].tolist()
            if len(positive_values) > 1:
                sum_values = sum(positive_values)
                greater_than_omega = [x >= omega for x in positive_values]
                if (sum_values >= tau) and (all(greater_than_omega)):
                    return True

            return False

        gold_arguments_names = [f'G{i}' for i in range(matrix.shape[0])]
        predicted_arguments_names = [f'P{i}' for i in range(matrix.shape[1])]

        categorization = { 'PM': 0, 'DISP': 0, 'SP': 0, 'MG': 0, 'MU': 0, 'UNR': 0 }

        for i, pred_arg in enumerate(predicted_arguments_names):
            column = matrix[:, i]
            positive_values = column[column > 0].tolist()
            if len(positive_values) == 0:
                categorization['MU'] += 1
            elif len(positive_values) == 1:
                if positive_values[0] == 1:
                    categorization['PM'] += 1
                else:
                    if is_split(matrix, column):
                        categorization['SP'] += 1
                    else:
                        categorization['DISP'] += 1

            elif len(positive_values) > 1:
                sum_values = sum(positive_values)
                greater_than_omega = [x >= omega for x in positive_values]

                if (sum_values >= tau) and (all(greater_than_omega)):
                    categorization['MG'] += 1
                else:
                    categorization['DISP'] += 1

        for i, gold_arg in enumerate(gold_arguments_names):
            row = matrix[i, :]
            positive_values = row[row > 0].tolist()
            if len(positive_values) == 0:
                categorization['UNR'] += 1

        return categorization
    
    
    def update_category_weights(self, current_epoch):
        
        self.matrix_errors[current_epoch, :] = self.acumulated_error_epoch[1:]
        
        if self.type_weighting == 1:
            
            if current_epoch > 0:
                current_row = self.matrix_errors[current_epoch, :]
                past_row = self.matrix_errors[current_epoch - 1, :]
                ratios = [c / p if p != 0 else c for c, p in zip(current_row, past_row)]
                exp_sum = sum(np.exp(wx) for wx in ratios)
                
                for i, wx in enumerate(ratios):
                    self.lambdas[i] = np.exp(wx) / exp_sum
                
            else:
                self.lambdas = [1, 1, 1, 1, 1]
            
          
        elif self.type_weighting == 2:
            
            if current_epoch > 0:
                current_row = self.matrix_errors[current_epoch, :]
                start_index = max(0, current_epoch - 3)
                past_row = np.mean(self.matrix_errors[start_index:current_epoch, :], axis=0)
                
                ratios = [c / p if p != 0 else c for c, p in zip(current_row, past_row)]
                exp_sum = sum(np.exp(wx) for wx in ratios)
                
                for i, wx in enumerate(ratios):
                    self.lambdas[i] = np.exp(wx) / exp_sum
            else:
                self.lambdas = [1, 1, 1, 1, 1]
            
          
        elif self.type_weighting == 3:
            current_row = self.matrix_errors[current_epoch, :]
            current_row_sum = np.sum(current_row)
            
            for i, wx in enumerate(current_row):
                self.lambdas[i] = wx / current_row_sum
        
        else:
            raise Exception

    def compute_loss(self, batch_logits, batch_labels, validation = False):
        
        # Auxiliary loss functions
        def simple_loss(ce_loss):
            return ce_loss
        
        def pm_guided_loss(ce_loss, num_pm, total_gold_arguments):
            pm_loss = 1 - (num_pm / total_gold_arguments)
            pm_loss.requires_grad_()
            return pm_loss
        
        def ce_pm_loss(ce_loss, num_errors_tensor, total_gold_arguments):
            pm_loss = 1 - (num_errors_tensor[0] / total_gold_arguments)
            return ce_loss + 1*pm_loss
        
        def ce_sum_errors_loss(ce_loss, num_errors_tensor):
            total_errors = torch.sum(num_errors_tensor)
            total_errors.requires_grad_()
            return ce_loss + 0.01*total_errors
        
        def ce_sum_weighted_errors(ce_loss, num_errors_tensor):
            weighted_sum = torch.sum(torch.stack([p*x for p,x in zip(self.initial_error_weights, num_errors_tensor)]))
            total_loss = ce_loss + weighted_sum
            return total_loss
        
        def ce_sum_weighted_errors_dyn(ce_loss, num_errors_tensor):
            weighted_sum = torch.sum(torch.stack([p*x for p,x in zip(self.lambdas, num_errors_tensor)]))
            total_loss = ce_loss + weighted_sum
            return total_loss
        
        ce_loss = self.base_loss_fn(batch_logits.view(-1, 3), batch_labels.view(-1).long())
        
        batch_predictions = torch.argmax(batch_logits, dim=2)
        
        instances = self.get_instances_from_batch(batch_predictions, batch_labels)
                
        num_items_per_category = collections.Counter( { 'PM': 0, 'DISP': 0, 'SP': 0, 'MG': 0, 'MU': 0, 'UNR': 0 })
        
        total_gold_arguments =  0
        
        for (labels, predictions) in instances:
            ground_truth_indices = self.get_arguments_indices(labels)
            predicted_args_indices = self.get_arguments_indices(predictions)
            
            R_matrix = self.compute_r_matrix(ground_truth_indices, predicted_args_indices)
            cat_counter = self.categorization_counter(R_matrix, 0.7, 0.35)
            
            num_items_per_category.update(cat_counter)
                    
            total_gold_arguments += len(ground_truth_indices)
            
        # Compute loss based on the difference between predicted and ground truth arguments
        num_errors_list = [num_items_per_category[x] for x in ['PM', 'DISP', 'SP', 'MG', 'MU', 'UNR']]
        num_errors_tensor = torch.tensor(num_errors_list, dtype=torch.float32)
        
        total_gold_arguments_tensor = torch.tensor([total_gold_arguments])
        
        if not validation:
            self.acumulated_error_epoch += np.array(num_errors_tensor)
        
        # Compute loss
        if self.config.error_type == 'simple':
            total_loss = simple_loss(ce_loss)
        elif self.config.error_type == 'pm-guided':
            total_loss = pm_guided_loss(ce_loss, num_errors_tensor[0], total_gold_arguments)
        elif self.config.error_type == 'ce-sum-errors':
            total_loss = ce_sum_errors_loss(ce_loss, num_errors_tensor[1:])
        elif self.config.error_type == 'ce-sum-weighted-errors':
            total_loss = ce_sum_weighted_errors(ce_loss, num_errors_tensor[1:])
        elif self.config.error_type == 'ce-sum-weighted-errors-dynamic':
            total_loss = ce_sum_weighted_errors_dyn(ce_loss, num_errors_tensor[1:])
        else:
            raise Exception("Incorrect loss type.")
        
        
        return total_loss

In [8]:
def train_model(model, train_loader, optimizer, loss_fn):

    model.train()

    train_loss = 0
    for batch in train_loader:
        batch = tuple(v.to(device) for t, v in batch.items())
        batch_input_ids, batch_token_type_ids, batch_attention_mask, batch_labels = batch
        
        loss, outputs = None, None
        
        logits = model(batch_input_ids, attention_mask = batch_attention_mask, token_type_ids = batch_token_type_ids, labels = batch_labels)
        

        loss = loss_fn.compute_loss(logits, batch_labels)

        train_loss += loss.item()

        # backprop
        optimizer.zero_grad()
        
        loss.backward()
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)

        optimizer.step()

    avg_train_loss = round((train_loss / len(train_loader)), 4)
    return avg_train_loss

def evaluate_model(model, dataloader, tokenizer, loss_fn):

    model.eval()

    eval_loss = 0
    eval_tokens, eval_labels, eval_predictions = [], [], []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = tuple(v.to(device) for t, v in batch.items())
            batch_input_ids, batch_token_type_ids, batch_attention_mask, batch_labels = batch
            logits = model(batch_input_ids, attention_mask = batch_attention_mask, token_type_ids = batch_token_type_ids, labels = batch_labels)
    
            loss = loss_fn.compute_loss(logits, batch_labels, validation = True)
            
            eval_loss += loss.item()
    
            batch_labels = batch_labels.detach().cpu().numpy()
            batch_predictions = np.argmax(logits.detach().cpu().numpy(), axis=2)
            
            clean_elements = clean_batch_elements(batch_input_ids, batch_labels, batch_predictions, tokenizer)
            
            eval_tokens += clean_elements[0]
            eval_labels += clean_elements[1]
            eval_predictions += clean_elements[2]
    
    flattened_labels = [j for sub in eval_labels for j in sub]
    flattened_predictions = [j for sub in eval_predictions for j in sub]
    
    eval_f1 = f1_score(flattened_labels, flattened_predictions, average = 'macro')
    return round((eval_loss / len(dataloader)), 4), eval_f1


def test_model(model, dataloader, tokenizer):

    model.eval()

    eval_tokens, eval_labels, eval_predictions = [], [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = tuple(v.to(device) for t, v in batch.items())
            batch_input_ids, batch_token_type_ids, batch_attention_mask, batch_labels = batch

            logits = model(batch_input_ids, token_type_ids = batch_token_type_ids, attention_mask = batch_attention_mask, labels = batch_labels)
    
            batch_labels = batch_labels.detach().cpu().numpy()
            batch_predictions = np.argmax(logits.detach().cpu().numpy(), axis=2)
            
            clean_elements = clean_batch_elements(batch_input_ids, batch_labels, batch_predictions, tokenizer)
            
            eval_tokens += clean_elements[0]
            eval_labels += clean_elements[1]
            eval_predictions += clean_elements[2]

    return eval_tokens, eval_labels, eval_predictions

In [ ]:
# loading data
dataname = 'pe' # datasets cited in the manuscript.
df = pd.read_csv(f'/kaggle/input/error-aware-argument-identification/{dataname}_sent.csv')
print(df.shape)

# cross-dataset testing
cross_tests = [x for x in ['pe', 'we', 'abstrct'] if x != dataname]

best_model = None
best_loss = float('inf')
best_run = 0

experiments_configuration = Config(runs = 10, epochs = 10, batch_size = 32, error_type = 'ce-sum-weighted-errors-dynamic', use_crf = False)

for nrun in range(experiments_configuration.runs):
    rs = generate_random_seed()
    set_random_seed(rs)
    
    run_best_eval_loss = float('inf')
    run_best_epoch = 0
    run_best_model_state = None
    
    train_loader, val_loader, test_loader = load_data(df, experiments_configuration)
    
    # Create model
    tagger = SimpleTagger(experiments_configuration)

    tagger.to(device)
    
    base_loss_fn = nn.CrossEntropyLoss()
    initial_category_weights = [1, 1, 1, 1, 1] # disp, sp, mg, mu, unr
    
    custom_loss = CustomLoss(base_loss_fn, initial_category_weights, experiments_configuration)
    
    optimizer = torch.optim.AdamW(tagger.parameters(), lr = 1e-4, eps = 1e-8)
    
    weights_info = []
    acummulated_errors_info = []
    train_eval_info = []
    
    # training and validation
    for epoch in range(experiments_configuration.epochs):
                
        train_loss = train_model(tagger, train_loader, optimizer, custom_loss)
        
        eval_loss, eval_f1 = evaluate_model(tagger, val_loader, experiments_configuration.tokenizer, custom_loss)
        
        print(f"Train_loss={train_loss} | Eval_loss={eval_loss} | Eval_F1={eval_f1}")
                        
        weights_info.append([epoch] + custom_loss.get_category_weights())
        acummulated_errors_info.append([epoch] + custom_loss.acumulated_error_epoch.tolist())
        train_eval_info.append([epoch, train_loss, eval_loss, eval_f1])
        
        if experiments_configuration.error_type == 'ce-sum-weighted-errors-dynamic':
            custom_loss.update_category_weights(epoch)
            
        custom_loss.restart_error_counter()        
        
        if eval_loss < run_best_eval_loss:
            run_best_eval_loss = eval_loss
            run_best_epoch = epoch
            run_best_model_state = copy.deepcopy(tagger.state_dict())
            print(f"New best model in epoch {epoch} (eval_loss = {eval_loss})")
            
    # testing
    best_tagger = SimpleTagger(experiments_configuration)
        
    best_tagger.load_state_dict(run_best_model_state)
    best_tagger.to(device)
    
    tokens, labels, preds = test_model(best_tagger, test_loader, experiments_configuration.tokenizer)
    assert len(tokens) == len(labels) == len(preds)
    if len(tokens) > 0:
        assert len(tokens[0]) == len(labels[0]) == len(preds[0])
        
    save_predictions(tokens, labels, preds, f'test_{nrun}_{dataname}.txt')
    
    ## cross-testing
    for test_dataname in cross_tests:
        cross_df = pd.read_csv(f'/kaggle/input/error-aware-argument-identification/{test_dataname}_sent.csv')
        _, _, cross_test_loader = load_data(cross_df, experiments_configuration)
        tokens, labels, preds = test_model(best_tagger, cross_test_loader, experiments_configuration.tokenizer)
        assert len(tokens) == len(labels) == len(preds)
        save_predictions(tokens, labels, preds, f'test_{nrun}_{dataname}_{test_dataname}.txt')
    
    if experiments_configuration.error_type == 'ce-sum-weighted-errors-dynamic':
        pd.DataFrame(weights_info, columns = ['epoch', 'disp', 'sp', 'mg', 'mu', 'unr']).to_csv(f'weights_{nrun}_{dataname}.csv', index = False)
    
    pd.DataFrame(acummulated_errors_info, columns = ['epoch', 'pm', 'disp', 'sp', 'mg', 'mu', 'unr']).to_csv(f'errors_{nrun}_{dataname}.csv', index = False)
    pd.DataFrame(train_eval_info, columns = ['epoch', 'train_loss', 'eval_loss', 'eval_f1']).to_csv(f'metrics_{nrun}_{dataname}.csv', index = False)
            
    if run_best_eval_loss < best_loss:
        best_loss = run_best_eval_loss
        best_model = copy.deepcopy(run_best_model_state)
        best_run = nrun
        
    print(f"end run {nrun}")
    print()
    
# saving best model
model_path = f"model-{best_run}-{dataname}.pt"
if best_model is not None:
    torch.save(best_model, model_path)

In [ ]:
import os
import zipfile
from IPython.display import FileLink

def zip_files(folder_path, zip_name):
    # Crear un archivo ZIP
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Recorrer todos los archivos en la carpeta
        for foldername, subfolders, filenames in os.walk(folder_path):
            for filename in filenames:
                # Comprobar si el archivo es un archivo TXT o CSV
                if filename.endswith('.txt') or filename.endswith('.csv') or filename.endswith('.pt'):
                    # Ruta completa del archivo
                    file_path = os.path.join(foldername, filename)
                    # Agregar el archivo al archivo ZIP
                    zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Llamar a la función para comprimir los archivos
folder_path = '/kaggle/working/'
zip_name = 'test.zip'
zip_files(folder_path, zip_name)

FileLink(r'test.zip')